Now import all necessary libraries

In [57]:
import math

import cv2
import numpy as np
import glob

Variable debug to see intermediate results

In [58]:
debug = False

Now we import camera matrix and distortion coefficient

Now we start to elaborate the video

In [67]:
import numpy as np
import cv2
import glob
import math

# get list of frames of video 1
listImages = glob.glob ( "framesVideo1/uncalibrated/*.jpg" )

# for each image
for image in listImages :
	
	# open the actual image in color mode
	actual_image = cv2.imread ( image , cv2.IMREAD_COLOR )
	
	# open the actual image in gray mode
	gray_actual_image = cv2.imread ( image , cv2.IMREAD_GRAYSCALE )
	
	# make binary threshold on gray actual image
	_ , threshold_gray_actual_image = cv2.threshold ( gray_actual_image , 190 , 255 , cv2.THRESH_BINARY )
	
	# save the threshold gray actual image
	cv2.imwrite ( image.replace ( "uncalibrated" , "binarythreshold" ) , threshold_gray_actual_image )
	
	# detect shape in threshold gray actual image using regions with same colors and intensity
	contours , hierarchy = cv2.findContours ( image = threshold_gray_actual_image , mode = cv2.RETR_TREE , method = cv2.CHAIN_APPROX_NONE )
	
	#save the index of actual contour
	index_actual_contour = 0
	
	# variables for detection of combination 100
	contours_100 = [ ]
	approx_100 = [ ]
	points_100 = [ ]
	
	# variables for detection of combination 110
	contours_102 = [ ]
	approx_102 = [ ]
	points_102 = [ ]
	
	# variables for detection of combination 112
	contours_112 = [ ]
	approx_112 = [ ]
	points_112 = [ ]
	
	# variables for detection of combination 112
	contours_110 = [ ]
	approx_110 = [ ]
	points_110 = [ ]
	
	# for each contour in threshold_gray_actual_image
	for contour in contours :
		
		# calculate the area of the actual region
		area_actual_region = cv2.contourArea ( contour )
		
		# approximate the shape of the actual polygon
		approx = cv2.approxPolyDP ( contour , 0.01 * cv2.arcLength ( contour , True ) , True )
		
		# if the area of the actual region is bigger than a minimum value
		if area_actual_region > 200 :
			
			# if we have 5 edges in the actual region
			if len ( approx ) == 5 :
				
				# calculate the perimeter of the contour
				perimeter = cv2.arcLength ( approx , True )
				
				# calculate the average of one edge
				avg_perimeter = perimeter / len ( approx )
				
				# save index of concave vertex
				first_edge_concave_vertex_clockwise = 0
				
				# initialize the list of children of actual polygon as empty
				children_contours = [ ]
				
				# we know that contours have form [next, previous, first child, parent]
				children = hierarchy [ 0 ] [ index_actual_contour ] [ 2 ]
				
				# while there are other childrens
				while children != -1 :
					
					# append actual children to the previous list
					children_contours.append ( children )
					
					# obtain the right contour
					children = hierarchy [ 0 ] [ children ] [ 0 ]
				
				# area children contours
				area_children_contours = [ ]
				
				# for each children contours
				for children_contour in children_contours :
					
					# calculate area actual children
					area_actual_children = cv2.contourArea ( contours [ children_contour ] )
					
					# save the actual area
					area_children_contours.append ( area_actual_children )
				
				# temp array to store the children contours
				temp_children_contours = children_contours.copy ( )
				
				# initialize as empty array the children contours
				children_contours = [ ]
				
				# for each area contour
				for i in range ( 0 , len ( area_children_contours ) ) :
					
					# if the actual area contour is greater than 0 and so it is a valid area
					if area_children_contours [ i ] > 20 :
						
						# save the actual children contour as valid
						children_contours.append ( temp_children_contours [ i ] )
				
				# convert the children contours list in numpy array
				children_contours = np.array ( children_contours )
				
				# if there are children in actual contour
				if len ( children_contours ) > 0 :
					
					# for each edge
					for i in range ( 0 , len ( approx ) ) :
						
						# calculate the distance between the previous and the next one
						previous = math.dist ( [ approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ] ] , [ approx [ (i - 1) % len ( approx ) ] [ 0 ] [ 0 ] , approx [ (i - 1) % len ( approx ) ] [ 0 ] [ 1 ] ] )
						next = math.dist ( [ approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ] ] , [ approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 0 ] , approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 1 ] ] )
						
						# if they are both less than average edge
						if previous < avg_perimeter and next < avg_perimeter :
							
							# this is the concave vertex
							first_edge_concave_vertex_clockwise = i
							
							# calculate the edge of the outer polygon
							second = approx [ (i + 2) % len ( approx ) ] [ 0 ]
							third = approx [ (i + 3) % len ( approx ) ] [ 0 ]
							
							# calculate the middle point of the previous edge
							half_second_third = [ (second [ 0 ] + third [ 0 ]) // 2 , (second [ 1 ] + third [ 1 ]) // 2 ]
							
							# calculate the middle point between the previous middle point and the concave vertex
							half_point_polygon = [ (approx [ first_edge_concave_vertex_clockwise ] [ 0 ] [ 0 ] + half_second_third [ 0 ]) // 2 , (approx [ first_edge_concave_vertex_clockwise ] [ 0 ] [ 1 ] + half_second_third [ 1 ]) // 2 ]
							
							# draw a circle in the previous middle point
							cv2.circle ( actual_image , (half_point_polygon [ 0 ] , half_point_polygon [ 1 ]) , 1 , (0 , 0 , 255) , -1 )
							
							# now draw circles of different color from the concave vertex in clockwise
							for j in range ( 0 , len ( approx ) ) :
								
								# if it is the concave vertex
								if j == 0 :
									
									# red circle
									actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (0 , 0 , 255) , thickness = -1 )
								
								# one vertex after the concave vertex
								elif j == 1 :
									
									# green circle
									actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (0 , 255 , 0) , thickness = -1 )
								
								# two vertex after the concave vertex
								elif j == 2 :
									
									# blue circle
									actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (255 , 0 , 0) , thickness = -1 )
								
								# three vertex after the concave vertex
								elif j == 3 :
									
									# yellow circle
									actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (0 , 255 , 255) , thickness = -1 )
								
								# four vertex after the concave vertex
								elif j == 4 :
									
									# pink circle
									actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (255 , 0 , 255) , thickness = -1 )
							
							# we can exit from the for of approx edges
							break
					
					# we can draw contours of the actual region but in the colored image
					cv2.drawContours ( actual_image , [ contour ] , 0 , (0 , 0 , 255) , 1 )
					
					# create a empty list to store all important points for each polygon
					points = [ ]
					
					# calculate the edge of the outer polygon
					second = approx [ (first_edge_concave_vertex_clockwise + 2) % len ( approx ) ] [ 0 ]
					third = approx [ (first_edge_concave_vertex_clockwise + 3) % len ( approx ) ] [ 0 ]
					
					# calculate the middle point of the previous edge
					half_second_third = [ (second [ 0 ] + third [ 0 ]) // 2 , (second [ 1 ] + third [ 1 ]) // 2 ]
					
					# calculate the middle point between the previous middle point and the concave vertex
					half_point_polygon = [ (approx [ first_edge_concave_vertex_clockwise ] [ 0 ] [ 0 ] + half_second_third [ 0 ]) // 2 , (approx [ first_edge_concave_vertex_clockwise ] [ 0 ] [ 1 ] + half_second_third [ 1 ]) // 2 ]
					
					# append to the list the outer point
					points.append ( half_second_third )
					
					# append to the list the middle point of the middle edge in the polygon
					points.append ( half_point_polygon )
					
					# append to the list the inner point
					points.append ( [ approx [ first_edge_concave_vertex_clockwise ] [ 0 ] [ 0 ] , approx [ first_edge_concave_vertex_clockwise ] [ 0 ] [ 1 ] ] )
					
					# for each children contour
					for single_children in children_contours :
						
						# draw contours of actual children
						cv2.drawContours ( actual_image , contours , single_children , (255 , 0 , 0) , 1 )
						
						# calculate the center of actual contour
						M = cv2.moments ( contours [ single_children ] )
						if M [ "m00" ] != 0 :
							cX = int ( M [ "m10" ] / M [ "m00" ] )
							cY = int ( M [ "m01" ] / M [ "m00" ] )
							
							# append to the list the central point of actual contour
							points.append ( [ cX , cY ] )
					
					# save the coordinates of the outer point
					x = points [ 0 ] [ 0 ]
					y = points [ 0 ] [ 1 ]
					
					# order list of points from outer to inner points through a line 2d
					points.sort ( key = lambda p : (p [ 0 ] - x) ** 2 + (p [ 1 ] - y) ** 2 )
					
					# counter for outer points
					counter_outer_points = 0
					
					# counter for central points
					counter_central_points = 0
					
					# counter for inner points
					counter_inner_points = 0
					
					# boolean to check if the central point is found
					check_center_point_found = False
					
					# for each point of polygon (no first because is the outer point, no last one because is the inner point)
					for i in range ( 1 , len ( points ) - 1 ) :
						
						# if the actual point is the central point of the polygon and is the first time that i found the central point
						if points [ i ] [ 0 ] == half_point_polygon [ 0 ] and points [ i ] [ 1 ] == half_point_polygon [ 1 ] and not check_center_point_found :
							
							# center point of the polygon found
							check_center_point_found = True
						
						# if it is a valid point to check
						else :
							
							# index for the correct contour
							index_correct_contour = 0
							
							# find the contour associated with the actual point
							for single_children in children_contours :
								
								# calculate the center of actual contour
								M = cv2.moments ( contours [ single_children ] )
								if M [ "m00" ] != 0 :
									cX = int ( M [ "m10" ] / M [ "m00" ] )
									cY = int ( M [ "m01" ] / M [ "m00" ] )
									
									# if the coordinates are correct
									if cX == points [ i ] [ 0 ] and cY == points [ i ] [ 1 ] :
										
										cv2.circle ( actual_image , (cX , cY) , 3 , (40 , 40 , 40) , -1 )
										
										#save the correct index for the contour
										index_correct_contour = single_children
										
										# exit from the for loop because the correct contour is found
										break
							
							# create correct data format to use forward the pointPolygonTest function
							list_array_points = contours [ index_correct_contour ].tolist ( )
							my_tuple = (int ( half_point_polygon [ 0 ] ) , int ( half_point_polygon [ 1 ] ))
							
							# if the center point is not just found
							if not check_center_point_found :
								
								# if the center point is in the actual contour point
								
								if cv2.pointPolygonTest ( np.array ( list_array_points ) , my_tuple , False ) == 1 :
									
									# increment central points
									counter_central_points = counter_central_points + 1
								
								# if the center point is not in the actual contour point
								else :
									
									# increment outer points
									counter_outer_points = counter_outer_points + 1
							
							# if the center point is found
							else :
								
								# if the center point is in the actual contour point
								if cv2.pointPolygonTest ( np.array ( list_array_points ) , my_tuple , False ) == 1 :
									
									# increment central points
									counter_central_points = counter_central_points + 1
								
								# if the center point is not in the actual contour point
								else :
									
									# increment inner points
									counter_inner_points = counter_inner_points + 1
					
					if counter_outer_points == 2 and counter_central_points == 1 and counter_inner_points == 2 :
						
						# for each children contour
						for single_children in children_contours :
							
							# draw contours of actual children
							cv2.drawContours ( actual_image , [ contour ] , 0 , (255 , 0 , 0) , 6 )
					
					if counter_outer_points == 2 and counter_central_points == 0 and counter_inner_points == 2 :
						
						# for each children contour
						for single_children in children_contours :
							
							# draw contours of actual children
							cv2.drawContours ( actual_image , [ contour ] , 0 , (0 , 255 , 0) , 6 )
					
					if counter_outer_points == 2 and counter_central_points == 0 and counter_inner_points == 0 :
						
						# for each children contour
						for single_children in children_contours :
							
							# draw contours of actual children
							cv2.drawContours ( actual_image , [ contour ] , 0 , (0 , 0 , 255) , 6 )
					
					if counter_outer_points == 1 and counter_central_points == 0 and counter_inner_points == 0 :
						
						contours_100.append ( [ contour ] )
						approx_100.append ( approx )
						points_100.append ( points )
					
					if counter_outer_points == 1 and counter_central_points == 0 and counter_inner_points == 2 :
						
						contours_102.append ( [ contour ] )
						approx_102.append ( approx )
						points_102.append ( points )
					
					if counter_outer_points == 1 and counter_central_points == 1 and counter_inner_points == 0 :
						
						contours_110.append ( [ contour ] )
						approx_110.append ( approx )
						points_110.append ( points )
					
					if counter_outer_points == 1 and counter_central_points == 1 and counter_inner_points == 2 :
						
						contours_112.append ( [ contour ] )
						approx_112.append ( approx )
						points_112.append ( points )
		
		# increment the index of actual contour
		index_actual_contour = index_actual_contour + 1
	
	# if there are two 100 in the image
	if len ( points_100 ) == 2 :
		
		# calculate the distance between their two points
		first = (points_100 [ 0 ] [ 0 ] [ 0 ] - points_100 [ 0 ] [ 1 ] [ 0 ]) ** 2 + (points_100 [ 0 ] [ 0 ] [ 1 ] - points_100 [ 0 ] [ 1 ] [ 1 ]) ** 2
		second = (points_100 [ 1 ] [ 0 ] [ 0 ] - points_100 [ 1 ] [ 1 ] [ 0 ]) ** 2 + (points_100 [ 1 ] [ 0 ] [ 1 ] - points_100 [ 1 ] [ 1 ] [ 1 ]) ** 2
		
		# if first distance is greater than the second one
		if first > second :
			cv2.drawContours ( actual_image , contours_100 [ 0 ] , 0 , (14 , 201 , 255) , 6 )
			cv2.drawContours ( actual_image , contours_100 [ 1 ] , 0 , (242 , 128 , 255) , 6 )
		else :
			cv2.drawContours ( actual_image , contours_100 [ 0 ] , 0 , (242 , 128 , 255) , 6 )
			cv2.drawContours ( actual_image , contours_100 [ 1 ] , 0 , (14 , 201 , 255) , 6 )
	
	# if there are two 102 in the image
	if len ( points_102 ) == 2 :
		
		# calculate the distance between their two points
		first = (points_102 [ 0 ] [ 0 ] [ 0 ] - points_102 [ 0 ] [ 1 ] [ 0 ]) ** 2 + (points_102 [ 0 ] [ 0 ] [ 1 ] - points_102 [ 0 ] [ 1 ] [ 1 ]) ** 2
		second = (points_102 [ 1 ] [ 0 ] [ 0 ] - points_102 [ 1 ] [ 1 ] [ 0 ]) ** 2 + (points_102 [ 1 ] [ 0 ] [ 1 ] - points_102 [ 1 ] [ 1 ] [ 1 ]) ** 2
		
		# if first distance is greater than the second one
		if first > second :
			cv2.drawContours ( actual_image , contours_102 [ 0 ] , 0 , (14 , 201 , 255) , 6 )
			cv2.drawContours ( actual_image , contours_102 [ 1 ] , 0 , (242 , 128 , 255) , 6 )
		else :
			cv2.drawContours ( actual_image , contours_102 [ 0 ] , 0 , (242 , 128 , 255) , 6 )
			cv2.drawContours ( actual_image , contours_102 [ 1 ] , 0 , (14 , 201 , 255) , 6 )
	
	# if there are two 110 in the image
	if len ( points_110 ) == 2 :
		
		# calculate the distance between their two points
		first = (points_110 [ 0 ] [ 0 ] [ 0 ] - points_110 [ 0 ] [ 1 ] [ 0 ]) ** 2 + (points_110 [ 0 ] [ 0 ] [ 1 ] - points_110 [ 0 ] [ 1 ] [ 1 ]) ** 2
		second = (points_110 [ 1 ] [ 0 ] [ 0 ] - points_110 [ 1 ] [ 1 ] [ 0 ]) ** 2 + (points_110 [ 1 ] [ 0 ] [ 1 ] - points_110 [ 1 ] [ 1 ] [ 1 ]) ** 2
		
		# if first distance is greater than the second one
		if first > second :
			cv2.drawContours ( actual_image , contours_110 [ 0 ] , 0 , (14 , 201 , 255) , 6 )
			cv2.drawContours ( actual_image , contours_110 [ 1 ] , 0 , (242 , 128 , 255) , 6 )
		else :
			cv2.drawContours ( actual_image , contours_110 [ 0 ] , 0 , (242 , 128 , 255) , 6 )
			cv2.drawContours ( actual_image , contours_110 [ 1 ] , 0 , (14 , 201 , 255) , 6 )
	
	# if there are two 112 in the image
	if len ( points_112 ) == 2 :
		
		# calculate the distance between their two points
		first = (points_112 [ 0 ] [ 0 ] [ 0 ] - points_112 [ 0 ] [ 1 ] [ 0 ]) ** 2 + (points_112 [ 0 ] [ 0 ] [ 1 ] - points_112 [ 0 ] [ 1 ] [ 1 ]) ** 2
		second = (points_112 [ 1 ] [ 0 ] [ 0 ] - points_112 [ 1 ] [ 1 ] [ 0 ]) ** 2 + (points_112 [ 1 ] [ 0 ] [ 1 ] - points_112 [ 1 ] [ 1 ] [ 1 ]) ** 2
		
		# if first distance is greater than the second one
		if first > second :
			cv2.drawContours ( actual_image , contours_112 [ 0 ] , 0 , (14 , 201 , 255) , 6 )
			cv2.drawContours ( actual_image , contours_112 [ 1 ] , 0 , (242 , 128 , 255) , 6 )
		else :
			cv2.drawContours ( actual_image , contours_112 [ 0 ] , 0 , (242 , 128 , 255) , 6 )
			cv2.drawContours ( actual_image , contours_112 [ 1 ] , 0 , (14 , 201 , 255) , 6 )
	
	cv2.imwrite ( image.replace ( "uncalibrated" , "polygonborders" ) , actual_image )


